# Progetto di social computing 2025
### Analisi di dataset di scopus


## Analisi dataset


In [3]:
import pandas as pd
import networkx as nx
import json
from networkx.readwrite import json_graph
from itertools import combinations

# =========================
# Carica dataset
# =========================
df = pd.read_csv("../data/nuovo_dataset.csv")

# Rimuovi righe senza autori o anno
df = df.dropna(subset=['Author full names', 'Year'])

# =========================
# Crea grafi per anno
# =========================
grafi_per_anno = {}

for anno in df['Year'].unique():
    G = nx.Graph()
    df_anno = df[df['Year'] == anno]

    for authors_str in df_anno['Author full names']:
        authors = [a.strip() for a in authors_str.split(';')]
        # crea arco tra ogni coppia di autori
        for u, v in combinations(authors, 2):
            if G.has_edge(u, v):
                G[u][v]['weight'] += 1
            else:
                G.add_edge(u, v, weight=1)
    grafi_per_anno[anno] = G

print("Grafi creati per anno:", list(grafi_per_anno.keys()))

# =========================
# Dizionari per risultati
# =========================
central_node_per_anno = {}
highlight_nodes_per_anno = {}
subgrafi_per_anno = {}
pos_per_anno = {}

# =========================
# Nodo centrale, sottografi e serializzazione JSON
# =========================
for anno, G in grafi_per_anno.items():
    if len(G) == 0:
        continue

    # Nodo centrale (grado massimo)
    central = max(G.degree, key=lambda x: x[1])[0]
    central_node_per_anno[anno] = central

    # Vicini equicentrali
    central_degree = G.degree(central)
    equicentral_neighbors = [
        n for n in G.neighbors(central)
        if G.degree(n) == central_degree
    ]
    highlight_nodes_per_anno[anno] = [central] + equicentral_neighbors

    # Sottografo centrale + vicini
    neighbors = list(G.neighbors(central))
    nodes = [central] + neighbors
    H = G.subgraph(nodes).copy()
    subgrafi_per_anno[anno] = H

    # Posizioni
    initial_pos = {central: (0, 0)}
    pos = nx.spring_layout(H, seed=42, pos=initial_pos, fixed=[central])
    pos_per_anno[anno] = pos

    # Serializzazione JSON
    data_json = json_graph.node_link_data(H)
    filename = f"../graphs/subgrafo_{anno}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data_json, f, ensure_ascii=False, indent=2)

print("Serializzazione dei sottografi completata!")


Grafi creati per anno: [np.int64(2025), np.int64(2024), np.int64(2023), np.int64(2022), np.int64(2021), np.int64(2020), np.int64(2019), np.int64(2018), np.int64(2017)]
Serializzazione dei sottografi completata!


In [2]:
df.columns

Index(['Authors', 'Author full names', 'Author(s) ID', 'Title', 'Year',
       'Source title', 'Volume', 'Cited by', 'DOI', 'Link', 'Affiliations',
       'Authors with affiliations', 'Abstract', 'Author Keywords',
       'Index Keywords', 'References', 'Correspondence Address', 'Publisher',
       'ISSN', 'Abbreviated Source Title', 'Document Type',
       'Publication Stage', 'EID'],
      dtype='object')

In [5]:
grafi_unico_json = {}

for anno in df['Year'].unique():
    G = nx.Graph()
    df_anno = df[df['Year'] == anno]

    for authors_str in df_anno['Author full names']:
        authors = [a.strip() for a in authors_str.split(';')]
        for u, v in combinations(authors, 2):
            if G.has_edge(u, v):
                G[u][v]['weight'] += 1
            else:
                G.add_edge(u, v, weight=1)

    if len(G) > 0:
        central = max(G.degree, key=lambda x: x[1])[0]
        central_degree = G.degree(central)
        equicentral_neighbors = [
            n for n in G.neighbors(central)
            if G.degree(n) == central_degree
        ]
        nodes = [central] + equicentral_neighbors
        H = G.subgraph(nodes).copy()
    else:
        H = G

    # ✅ Chiave convertita in stringa
    grafi_unico_json[str(anno)] = json_graph.node_link_data(H)

# Salva in un unico JSON
with open("../graphs/grafi_tutti.json", "w", encoding="utf-8") as f:
    json.dump(grafi_unico_json, f, ensure_ascii=False, indent=2)

print("Tutti i sottografi salvati in un unico file: grafi_tutti.json")


Tutti i sottografi salvati in un unico file: grafi_tutti.json


In [6]:
import json
from networkx.readwrite import json_graph
import networkx as nx

# Carica JSON
with open("../graphs/grafi_tutti.json", "r", encoding="utf-8") as f:
    dati = json.load(f)

# Esempio: ricostruire il grafo del 2020
G2020 = json_graph.node_link_graph(dati["2020"])
print(G2020.nodes(), G2020.edges())


['Gigli, Gian Luigi (7101781977)'] []
